In [1]:
%reset -f
import sys
sys.path.insert(0,'..')
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.utils import to_categorical
from wavhandler import *
from utils_train import train_test_val_split

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
Using TensorFlow backend.


In [3]:
data1 = Dataset('Leafminers')
data1.read(data=data1.target_classes[0], setting='psd_dB', labels='text')

data2 = Dataset('LG')
data2.read(data=data2.target_classes[0], setting='psd_dB', labels='text')

data3 = Dataset('LG')
data3.read(data=data3.target_classes[1], setting='psd_dB', labels='text')

data4 = Dataset('Pcfruit')
data4.read(data=data4.target_classes[1], setting='psd_dB', labels='text')

data1.clean(plot=False)
data2.clean(plot=False)
data3.clean(plot=False)
data4.clean(plot=False)

data = pd.DataFrame()
data['fnames'] = pd.concat([data1.filenames, data2.filenames, data3.filenames, data4.filenames], axis=0).reset_index(drop=True)
data['y'] = big.fnames.apply(lambda x: x.split('/')[6])
# big.dropna(how='any', axis=1, inplace=True)
data.sample(10)

Data: Chicorii.
Read 1451 filenames in 0.30 seconds.
Loaded data into matrix in 1.59 seconds.
Data: LG_drosophila_10_09.
Read 5536 filenames in 0.31 seconds.
Loaded data into matrix in 3.26 seconds.
Data: LG_zapr_26_09.
Read 7210 filenames in 0.10 seconds.
Loaded data into matrix in 4.96 seconds.
Data: D. suzukii.
Read 6423 filenames in 0.30 seconds.
Loaded data into matrix in 11.82 seconds.


,fnames,y
20223,/home/kalfasyan/data/insects/Pcfruit/D. suzuki...,D. suzukii
9834,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09
3552,/home/kalfasyan/data/insects/LG/LG_drosophila_...,LG_drosophila_10_09
4482,/home/kalfasyan/data/insects/LG/LG_drosophila_...,LG_drosophila_10_09
2542,/home/kalfasyan/data/insects/LG/LG_drosophila_...,LG_drosophila_10_09
8998,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09
20105,/home/kalfasyan/data/insects/Pcfruit/D. suzuki...,D. suzukii
17733,/home/kalfasyan/data/insects/Pcfruit/D. suzuki...,D. suzukii
11500,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09
16284,/home/kalfasyan/data/insects/Pcfruit/D. suzuki...,D. suzukii


In [29]:
X,y = data.fnames.values.reshape(-1,1), data.y.values

In [30]:
# data = Dataset('Wingbeats')
# data.read(data="all", setting='read', loadmat=False, labels='nr')

In [31]:
# X,y = data.filenames.values.reshape(-1,1), data.y.values

model_name='first_' 
setting='raw'

In [32]:
top_weights_path = TEMP_DATADIR + str(model_name) + '.h5'
targets = len(np.unique(y))

if setting == 'psd_dB':
    X = transform_data(X, setting=setting)

X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,y, random_state=0)

# # Convert label to onehot
# y_train = to_categorical(y_train, num_classes=targets)
# y_val = to_categorical(y_val, num_classes=targets)
# y_test = to_categorical(y_test, num_classes=targets)

# X_train = np.expand_dims(X_train, axis=-1)
# X_val = np.expand_dims(X_val, axis=-1)
# X_test = np.expand_dims(X_test, axis=-1)

X_train shape: 	(14846, 1)
X_test shape:	(2062, 1)
X_val shape:	(3712, 1)



## MODEL

In [33]:
# Build the Neural Network
model = Sequential()

model.add(Conv1D(16, 3, activation='relu', input_shape=(5000, 1)))
model.add(Conv1D(16, 3, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(Conv1D(256, 3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())

model.add(Dropout(0.5))
model.add(Dense(targets, activation='softmax'))

## Compile and fit

In [34]:
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

callbacks_list = [ModelCheckpoint(top_weights_path, monitor = 'val_acc', verbose = 1, save_best_only = True, save_weights_only = True),
    EarlyStopping(monitor = 'val_acc', patience = 6, verbose = 1),
    ReduceLROnPlateau(monitor = 'val_acc', factor = 0.1, patience = 3, verbose = 1),
    CSVLogger('model_' + str(model_name) + '.log')]


In [35]:
import math
from utils_train import train_generator, valid_generator
target_names = list(np.unique(data.y))
batch_size = 32
model.fit_generator(train_generator(X_train, y_train, batch_size=32,
                                   target_names=target_names,
                                   setting=setting),
                   steps_per_epoch = int(math.ceil(float(len(X_train)) / float(batch_size))),
                   epochs=100,
                   validation_data = valid_generator(X_val, y_val,
                                                    batch_size=32,
                                                    target_names=target_names,
                                                    setting=setting),
                    validation_steps=int(math.ceil(float(len(X_test))/float(batch_size))),
                    callbacks = callbacks_list)

Instructions for updating:
Use tf.cast instead.


Epoch 1/100


TypeError: expected str, bytes or os.PathLike object, not numpy.ndarray

## Test model

In [9]:
model.load_weights(top_weights_path)
loss, acc = model.evaluate_generator(valid_generator(X_test, 
                                                    y_test, 
                                                    batch_size=batch_size, 
                                                    setting=setting, 
                                                    target_names=target_names),
        steps = int(math.ceil(float(len(X_test)) / float(batch_size))))

print('loss', loss)
print('Test accuracy:', acc)

from keras.models import model_from_yaml
# serialize model to YAML
model_yaml = model.to_yaml()
with open(TEMP_DATADIR + model_name + "_raw_final.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

loss 3.210132122039795
Test accuracy: 0.10000000149011612
